In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phishing-website-detector-based-on-url/verified_online.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/Phishing.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/Final Report.pdf
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/Defacement.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/Spam.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/Malware.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/Malware_dataset.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/DefacementSitesURLFiltered.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/spam_dataset.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/phishing_dataset.csv
/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/Benign_list_big_final.csv


In [2]:
!pip install python-whois
!pip install dnspython
!pip install beautifulsoup4
!pip install requests
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103263 sha256=cd0aee239161d82049159a6185eff68ad7b26372028a7acde5d52d1e56101d06
  Stored in directory: /root/.cache/pip/wheels/ae/7d/a1/7cfba90ff34974c64149b70f689ff77dde232b8f1ec5de43b3
Successfully built python-whois
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 5.7 MB/s eta 0:00:00


In [3]:
total_target = 12000
records_from_dataset = 4000
start_from = 8000
end_at = start_from + records_from_dataset

In [4]:
data0 = pd.read_csv('/kaggle/input/phishing-website-detector-based-on-url/verified_online.csv')
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12489 entries, 0 to 12488
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   phish_id           12489 non-null  int64 
 1   url                12489 non-null  object
 2   phish_detail_url   12489 non-null  object
 3   submission_time    12489 non-null  object
 4   verified           12489 non-null  object
 5   verification_time  12489 non-null  object
 6   online             12489 non-null  object
 7   target             12489 non-null  object
dtypes: int64(1), object(7)
memory usage: 780.7+ KB


In [5]:
#Collecting 12,000 Phishing URLs randomly
phishurl = data0.sample(n = total_target, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7735364,https://bpnote0010.web.app/,http://www.phishtank.com/phish_detail.php?phis...,2022-09-07T23:32:15+00:00,yes,2022-09-07T23:42:55+00:00,yes,Other
1,7658848,https://stepp8-c4298.firebaseapp.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-08-17T01:09:26+00:00,yes,2022-08-17T01:22:12+00:00,yes,Other
2,7647742,https://samrockchem.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-08-11T14:24:55+00:00,yes,2022-08-11T14:32:26+00:00,yes,Other
3,7991817,https://eurobank75.godaddysites.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-01-02T02:35:00+00:00,yes,2023-01-02T02:42:42+00:00,yes,Other
4,7991940,https://chime35.godaddysites.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-01-02T02:45:46+00:00,yes,2023-01-02T02:52:32+00:00,yes,Other


In [6]:
#Loading legitimate files 
data1 = pd.read_csv("/kaggle/input/identifying-suspiciousurlsmllexical-analysis/URL/Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35377 entries, 0 to 35376
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URLs    35377 non-null  object
dtypes: object(1)
memory usage: 276.5+ KB


In [7]:
#Collecting 12,000 Legitimate URLs randomly
legiurl = data1.sample(n = total_target, random_state = 4).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://mic.com/articles/117166/meet-marilyn-mo...
1,http://twitter.com/home?status=%E3%83%8C%E3%81...
2,https://privatbank.ua/ua/business/srednemu-i-k...
3,http://themeforest.net/item/amanda-a-responsiv...
4,http://getpocket.com/signup?mode=socialmode&ro...


In [8]:
# Exraction of features from the URL
#  0   having_IP_Address
#  1   URL_Length
#  2   Shortining_Service
#  3   having_At_Symbol
#  4   double_slash_redirecting
#  5   Prefix_Suffix
#  6   having_Sub_Domain
#  7   URL_Depth
#  8   Domain_registeration_length
#  9   Favicon
#  10  port
#  11  HTTPS_token
#  12  Request_URL
#  13  URL_of_Anchor
#  14  Links_in_tags
#  15  SFH
#  16  Submitting_to_email
#  17  Abnormal_URL
#  18  Redirect
#  19  on_mouseover
#  20  RightClick
#  21  popUpWidnow
#  22  Iframe
#  23  age_of_domain
#  24  DNSRecord
#  25  web_traffic

# Above fetatures function returns
# 1 if the URL is Phishing,
# -1 if the URL is Legitimate and
# 0 if the URL is Suspicious

import re
import whois
import datetime
import requests
import ipaddress
from dns import resolver
from bs4 import BeautifulSoup
from urllib.parse import urlparse


class FeatureExtraction:
    def __init__(self, url):
        self.url = url
        self.parsedurl = urlparse(self.url)
        self.domain = self.parsedurl.netloc

        try:
            self.whois = whois.whois(self.domain)
        except:
            self.whois = None

        try:
            self.request = requests.get(self.url, timeout=5, headers={
                                        "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
            self.soup = BeautifulSoup(self.request.content, 'html.parser')
        except:
            self.request = None
            self.soup = None

        self.shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
            r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
            r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
            r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
            r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
            r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
            r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
            r"tr\.im|link\.zip\.net"

    def getFeaturesDict(self):
        return {
            "having_IP_Address": self.having_IP_Address(),
            "URL_Length": self.URL_Length(),
            "Shortining_Service": self.Shortining_Service(),
            "having_At_Symbol": self.having_At_Symbol(),
            "double_slash_redirecting": self.double_slash_redirecting(),
            "Prefix_Suffix": self.Prefix_Suffix(),
            "having_Sub_Domain": self.having_Sub_Domain(),
            "URL_Depth": self.URL_Depth(),
            "Domain_registeration_length": self.Domain_registeration_length(),
            "Favicon": self.Favicon(),
            "port": self.port(),
            "HTTPS_token": self.HTTPS_token(),
            "Request_URL": self.Request_URL(),
            "URL_of_Anchor": self.URL_of_Anchor(),
            "Links_in_tags": self.Links_in_tags(),
            "SFH": self.SFH(),
            "Submitting_to_email": self.Submitting_to_email(),
            "Abnormal_URL": self.Abnormal_URL(),
            "Redirect": self.Redirect(),
            "on_mouseover": self.on_mouseover(),
            "RightClick": self.RightClick(),
            "popUpWidnow": self.popUpWidnow(),
            "Iframe": self.Iframe(),
            "age_of_domain": self.age_of_domain(),
            "DNSRecord": self.DNSRecord(),
            "web_traffic": self.web_traffic()
        }

    """#### ** IP Address in the URL **
    Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.
    If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def having_IP_Address(self):
        try:
            ipaddress.ip_address(self.domain)
            return 1
        except:
            return -1

    """#### ** Length of URL **
    Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.
    If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (suspicious) else -1 (legitimate).
    """

    def URL_Length(self):
        if len(self.url) < 54:
            return -1
        elif len(self.url) >= 54 and len(self.url) <= 75:
            return 0
        else:
            return 1

    """#### ** Using URL Shortening Services “TinyURL” **
    URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL. 
    If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Shortining_Service(self):
        if re.search(self.shortening_services, self.url):
            return 1
        else:
            return -1

    """#### ** "@" Symbol in URL **
    Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol. 
    If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def having_At_Symbol(self):
        if '@' in self.url:
            return 1
        else:
            return -1

    """#### ** Redirection "//" in URL **
    Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed.
    If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def double_slash_redirecting(self):
        if re.search(r'https?://[^\s]*//', self.url):
            return 1
        else:
            return -1

    """#### ** Prefix or Suffix "-" in Domain **
    Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage. 
    If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Prefix_Suffix(self):
        if '-' in self.domain:
            return 1
        else:
            return -1

    """#### ** SubDomains **    
    If the URL has more than 2 subdomains, the value assigned to this feature is 1 (phishing) or else 0 (suspicious) else -1 (legitimate).
    """

    def having_Sub_Domain(self):
        count = self.domain.count('.')
        if count <= 2:
            return -1
        elif count > 2 and count <= 3:
            return 0
        else:
            return 1

    """#### ** Depth of URL **
    Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.
    The value of feature is a numerical based on the URL.
    """

    def URL_Depth(self):
        depth = 0
        subdirs = self.parsedurl.path.split('/')
        for subdir in subdirs:
            if subdir:
                depth += 1
        return depth

    """#### ** End Period of Domain **
    This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or more for this project. 
    If end period of domain < 6 months, the vlaue of this feature is 1 (phishing) else -1 (legitimate).
    """

    def Domain_registeration_length(self):
        if self.whois is None:
            return 1

        try:
            if type(self.whois['expiration_date']) is list:
                expiration_date = self.whois['expiration_date'][0]
            else:
                expiration_date = self.whois['expiration_date']

            registration_length = abs(
                (expiration_date - datetime.datetime.now()).days)
            if registration_length / 30 >= 6:
                return -1
            else:
                return 1
        except:
            return 1

    """#### ** Favicon **
    Checks for the presence of favicon in the website. The presence of favicon in the website can be used as a feature to detect phishing websites.
    If the website has favicon, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Favicon(self):
        try:
            if re.findall(r'favicon', self.soup.text) or \
                    self.soup.find('link', rel='shortcut icon') or \
                    self.soup.find('link', rel='icon'):
                return -1
            else:
                return 1
        except:
            return 1

    """#### ** Non-Standard Port **
    Checks for the use of non-standard port. Phishers often use non-standard ports in the URL in order to make it look like a legitimate one.
    If the URL uses non-standard port, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def port(self):
        if self.parsedurl.port:
            return 1
        else:
            return -1

    """#### ** "http/https" in Domain name **
    Checks for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.
    If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def HTTPS_token(self):
        if 'https' in self.domain:
            return 1
        else:
            return -1

    """### ** Request_URL **
    The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times.
    """

    def Request_URL(self):
        try:
            if len(self.request.history) <= 1:
                return -1
            elif len(self.request.history) <= 3:
                return 0
            else:
                return 1
        except:
            return -1

    """#### ** URL_of_Anchor **
    The presence of “<a>” HTML tag in the URL is a strong indicator of phishing websites. This feature checks for the presence of “<a>” tag in the URL.
    If the URL has “<a>” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def URL_of_Anchor(self):
        try:
            count = 0
            for i in self.soup.find_all('a'):
                if i.has_attr('href'):
                    count += 1
            if count == 0:
                return 1
            else:
                return -1
        except:
            return 1

    """#### ** Links_in_tags **
    The presence of “<link>” HTML tag in the URL is a strong indicator of phishing websites. This feature checks for the presence of “<link>” tag in the URL.
    If the URL has “<link>” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Links_in_tags(self):
        try:
            count = 0
            for i in self.soup.find_all('link'):
                if i.has_attr('href'):
                    count += 1
            if count == 0:
                return 1
            else:
                return -1
        except:
            return 1

    """#### ** SFH **
    The presence of “<form>” HTML tag in the URL is a strong indicator of phishing websites. This feature checks for the presence of “<form>” tag in the URL.
    If the URL has “<form>” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def SFH(self):
        try:
            if self.soup.find('form'):
                return 1
            else:
                return -1
        except:
            return 0

    """#### ** Submitting_to_email **
    The presence of “mailto:” in the URL is a strong indicator of phishing websites. This feature checks for the presence of “mailto:” in the URL.
    If the URL has “mailto:” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Submitting_to_email(self):
        try:
            if self.soup.find('mailto:'):
                return 1
            else:
                return -1
        except:
            return 0

    """#### ** Abnormal_URL **
    The presence of “<script>” HTML tag in the URL is a strong indicator of phishing websites. This feature checks for the presence of “<script>” tag in the URL.
    If the URL has “<script>” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Abnormal_URL(self):
        try:
            if re.findall(r'script|javascript|alert|onmouseover|onload|onerror|onclick|onmouse', self.url):
                return 1
            else:
                return -1
        except:
            return -1

    """#### ** Redirect **
    The presence of “<meta>” HTML tag in the URL is a strong indicator of phishing websites. This feature checks for the presence of “<meta>” tag in the URL.
    If the URL has “<meta>” tag, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Redirect(self):
        try:
            if self.soup.find('meta', attrs={'http-equiv': 'refresh'}):
                return 1
            else:
                return -1
        except:
            return -1

    """### **3.3.2. Status Bar Customization**
    Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the “onMouseOver” event, and check if it makes any changes on the status bar
    If the response is empty or onmouseover is found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).
    """

    def on_mouseover(self):
        try:
            if re.findall(r"onmouseover", self.soup.text):
                return 1
            else:
                return -1
        except:
            return -1

    """### ** Disabling Right Click **
    Phishers use JavaScript to disable the right-click function, so that users cannot view and save the webpage source code. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “event.button==2” in the webpage source code and check if the right click is disabled.
    If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).
    """

    def RightClick(self):
        try:
            if re.findall(r"contextmenu|event.button ?== ?2", self.soup.text):
                return 1
            else:
                return -1
        except:
            return -1

    """### ** PopUp Window **
    Phishers may use JavaScript to open a fake webpage in a new window to trick users. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “window.open” in the webpage source code and check if the pop-up window is opened.
    If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).
    """

    def popUpWidnow(self):
        try:
            if re.findall(r"alert\(|onMouseOver|window.open", self.soup.text):
                return 1
            else:
                return -1
        except:
            return -1

    """### ** IFrame Redirection **
    IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation. 
    If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def Iframe(self):
        try:
            if re.findall(r"[<iframe>|<frameBorder>]", self.soup.text):
                return 1
            else:
                return -1
        except:
            return -1

    """#### ** Age of Domain **
    This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.
    If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else -1 (legitimate).
    """

    def age_of_domain(self):
        if self.whois is None:
            return 1

        try:
            if type(self.whois['creation_date']) is list:
                creation_date = self.whois['creation_date'][0]
            else:
                creation_date = self.whois['creation_date']

            ageofdomain = abs((datetime.datetime.now() - creation_date).days)
            if ageofdomain / 30 > 12:
                return -1
            else:
                return 1
        except:
            return 1

    """#### ** DNS Record **
    For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname. 
    If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else -1 (legitimate).
    """

    def DNSRecord(self):
        try:
            resolver.resolve(self.domain, 'A')
            return -1
        except:
            return 1

    """#### ** Web Traffic **
    This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.
    If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else -1 (legitimate).
    """

    def web_traffic(self):
        try:
            alexadata = BeautifulSoup(requests.get(
                "http://data.alexa.com/data?cli=10&dat=s&url=" + self.domain, timeout=10).content, 'lxml')
            rank = int(alexadata.find('reach')['rank'])
            if rank < 100000:
                return -1
            else:
                return 1
        except:
            return 1


In [9]:
#Function to extract features
def extraction(url,label):
    obj = FeatureExtraction(url)
    result = dict(Url = obj.url, Label = label)
    result.update(obj.getFeaturesDict())

    return result

In [10]:
def get_features(list_url, label):
    #Extracting the feautres & storing them in a list
    text_label = 'phished' if label else 'legitimate'
    features = []
    
    for i in range(start_from, end_at):
        output = extraction(list_url[i], label)
        features.append(output)
        print('Processed', text_label, 'url :', i)
    return features

In [11]:
#converting the list to dataframe
legi_features = get_features(legiurl['URLs'], 0)
legitimate = pd.DataFrame.from_dict(legi_features)

/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


Processed legitimate url : 8000
Processed legitimate url : 8001
Processed legitimate url : 8002
Processed legitimate url : 8003
Processed legitimate url : 8004
Processed legitimate url : 8005
Processed legitimate url : 8006
Processed legitimate url : 8007
Processed legitimate url : 8008
Processed legitimate url : 8009
Processed legitimate url : 8010
Processed legitimate url : 8011
Processed legitimate url : 8012
Processed legitimate url : 8013
Processed legitimate url : 8014
Processed legitimate url : 8015
Processed legitimate url : 8016
Processed legitimate url : 8017
Processed legitimate url : 8018
Processed legitimate url : 8019
Processed legitimate url : 8020
Processed legitimate url : 8021
Processed legitimate url : 8022
Processed legitimate url : 8023
Processed legitimate url : 8024
Processed legitimate url : 8025
Processed legitimate url : 8026
Processed legitimate url : 8027
Processed legitimate url : 8028
Processed legitimate url : 8029
Processed legitimate url : 8030
Processe

In [12]:
legitimate.head()

,Url,Label,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,URL_Depth,...,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic
0,http://superuser.com/questions/862530/3-pin-cp...,0,-1,1,-1,-1,-1,-1,-1,3,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
1,https://www.gov.uk/government/organisations/ci...,0,-1,1,-1,-1,-1,-1,-1,3,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
2,http://emgn.com/tv/watch-24-people-making-stat...,0,-1,1,-1,-1,-1,-1,-1,2,...,0,-1,-1,-1,-1,-1,-1,-1,1,1
3,http://slashdot.org/story/14/08/01/1643234/ins...,0,-1,1,-1,-1,-1,-1,-1,6,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
4,http://buzzfil.net/article/5391/animaux/15-ill...,0,-1,1,-1,-1,-1,-1,-1,4,...,0,-1,-1,-1,-1,-1,-1,-1,-1,1


In [13]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('legitimate.csv', index= False)


In [14]:
#Extracting the feautres & storing them in a list
phish_features = get_features(phishurl['url'], 1)
phishing = pd.DataFrame.from_dict(phish_features)

/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


Processed phished url : 8000
Processed phished url : 8001
Processed phished url : 8002
Processed phished url : 8003
Processed phished url : 8004
Processed phished url : 8005
Processed phished url : 8006
Processed phished url : 8007
Processed phished url : 8008
Processed phished url : 8009
Processed phished url : 8010
Processed phished url : 8011
Processed phished url : 8012
Processed phished url : 8013
Processed phished url : 8014
Processed phished url : 8015
Processed phished url : 8016
Processed phished url : 8017
Processed phished url : 8018
Processed phished url : 8019
Processed phished url : 8020
Processed phished url : 8021
Processed phished url : 8022
Processed phished url : 8023
Processed phished url : 8024
Processed phished url : 8025
Processed phished url : 8026
Processed phished url : 8027
Processed phished url : 8028
Processed phished url : 8029
Processed phished url : 8030
Processed phished url : 8031
Processed phished url : 8032
Processed phished url : 8033
Processed phis

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


Processed phished url : 9270
Processed phished url : 9271
Processed phished url : 9272
Processed phished url : 9273
Processed phished url : 9274
Processed phished url : 9275
Processed phished url : 9276
Processed phished url : 9277
Processed phished url : 9278
Processed phished url : 9279
Processed phished url : 9280
Processed phished url : 9281
Processed phished url : 9282
Processed phished url : 9283
Processed phished url : 9284
Processed phished url : 9285
Processed phished url : 9286
Processed phished url : 9287
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Processed phished url : 9288
Processed phished url : 9289
Processed phished url : 9290
Processed phished url : 9291
Processed phished url : 9292
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Processed phished url : 9293
Processed phished url : 9294
Processed phished url : 9295
Processed phished url : 9296
Processed phished url : 9297
Error tryi

In [15]:
phishing.head()

,Url,Label,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,URL_Depth,...,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic
0,https://sultan-raza.github.io/Netflix-Clone,1,-1,-1,-1,-1,-1,1,-1,1,...,-1,-1,-1,-1,-1,-1,1,1,-1,1
1,https://duelbotsnftmint.netlify.app/,1,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,1,1,-1,1
2,https://dejpaad.com:2096/cpsess4599319404/3rdp...,1,-1,0,-1,-1,-1,-1,-1,3,...,0,-1,-1,-1,-1,-1,-1,1,1,1
3,https://s.yam.com/oXYqp,1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
4,https://www.df-xiangfa.com/jp,1,-1,-1,-1,-1,-1,1,-1,1,...,0,-1,-1,-1,-1,-1,-1,-1,-1,1


In [16]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('phishing.csv', index= False)

In [17]:
#Concatenating the dataframes into one 
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Url,Label,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,URL_Depth,...,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic
0,http://superuser.com/questions/862530/3-pin-cp...,0,-1,1,-1,-1,-1,-1,-1,3,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
1,https://www.gov.uk/government/organisations/ci...,0,-1,1,-1,-1,-1,-1,-1,3,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
2,http://emgn.com/tv/watch-24-people-making-stat...,0,-1,1,-1,-1,-1,-1,-1,2,...,0,-1,-1,-1,-1,-1,-1,-1,1,1
3,http://slashdot.org/story/14/08/01/1643234/ins...,0,-1,1,-1,-1,-1,-1,-1,6,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
4,http://buzzfil.net/article/5391/animaux/15-ill...,0,-1,1,-1,-1,-1,-1,-1,4,...,0,-1,-1,-1,-1,-1,-1,-1,-1,1


In [18]:
urldata.tail()

,Url,Label,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,URL_Depth,...,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic
7995,https://yahoo-103496.weeblysite.com/,1,-1,-1,-1,-1,-1,1,-1,0,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
7996,https://laubros.com:2096/cpsess6807296530/3rdp...,1,-1,0,-1,-1,-1,-1,-1,3,...,-1,-1,-1,-1,-1,-1,1,1,1,1
7997,https://eejy91.webwave.dev/,1,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
7998,https://w12netempresasbrdsco.rds.land/brdsconet12,1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
7999,https://accepaccound.start.page/,1,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [19]:
print(urldata.shape)
# Storing the data in CSV file
urldata.to_csv('dataset.csv', index=False)
     

(8000, 28)
